In [1]:
import os
import requests
import datetime
import pandas as pd
import psycopg

from joblib import load, dump
from tqdm import tqdm
from loguru import logger

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error


In [2]:
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ( ColumnDriftMetric, DatasetDriftMetric, 
    DatasetMissingValuesMetric, ColumnQuantileMetric )


In [3]:
files = [
    ('green_tripdata_2024-01.parquet', './data'),
    ('green_tripdata_2024-02.parquet', './data'),
    ('green_tripdata_2024-03.parquet', './data'),
#    ('green_tripdata_2024-04.parquet', './data'),
    ]


In [4]:
print(" [i] Download datasets..")
for file, path in files:
    save_path=f"{path}/{file}"
    if os.path.exists(save_path): continue

    url=f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file}"
    resp=requests.get(url, stream=True)
    with open(save_path, "wb") as handle:
        for data in tqdm(resp.iter_content(),
                        desc=f"{file}",
                        postfix=f"save to {save_path}",
                        total=int(resp.headers["Content-Length"])):
            handle.write(data)
            

 [i] Download datasets..


In [5]:
# load dataset and model
raw_dataset = pd.read_parquet('data/green_tripdata_2024-03.parquet')
ref_dataset = pd.read_parquet('data/reference.parquet')

with open('models/lin_reg.bin', 'rb') as f_input:
    model = load(f_input)


In [6]:
# features
predict = "duration"
numeric_features = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]
categorical_features = ["PULocationID", "DOLocationID"]


In [7]:
# column mapping
column_mapping = ColumnMapping(
    target=None,
    prediction='prediction',
    numerical_features=numeric_features,
    categorical_features=categorical_features
)


In [8]:
# report
report = Report(
    metrics=[
        ColumnDriftMetric(column_name='prediction'),
        ColumnQuantileMetric(column_name='fare_amount', quantile=0.5),
    ]
)


In [9]:
# filters
## lpep_pickup_datetime
## raw_dataset['lpep_pickup_datetime'].dt.year == 2024
## taw_dataset['lpep_pickup_datetime'].dt.month == 3
raw_dataset = raw_dataset[ raw_dataset['lpep_pickup_datetime'].dt.year == 2024 ]
raw_dataset = raw_dataset[ raw_dataset['lpep_pickup_datetime'].dt.month == 3 ]


In [10]:
# dates, so we don't have to calculate them
dates = list(pd.to_datetime(raw_dataset['lpep_pickup_datetime']).dt.date.unique())


In [11]:
logger.info(raw_dataset.shape)


2024-06-21 12:41:30.704 | INFO     | __main__:<module>:1 - (57447, 20)


In [12]:

def prep_dataset(dff: pd.DataFrame) -> pd.DataFrame:
    """
    Create a "y_val" (or "predicted") column named "duration",
    and remove outliers from the dataset
    """

    # create "duration" (aka target)
    dff["duration"] = dff.lpep_dropoff_datetime - dff.lpep_pickup_datetime
    dff["duration"] = dff["duration"].apply(lambda td : float(td.total_seconds())/60)

    # filter out outliers
    dff = dff[ (dff['duration'] >= 0) & (dff['duration'] <= 60) ]
    dff = dff[ (dff['passenger_count'] > 0) & (dff['passenger_count'] <= 8) ]

    return dff


In [13]:

def calculate_metric(df: pd.DataFrame):
    """
    Calculate the metrics for the "metric" above.
    """

    # logger.debug(f" [i] Running model prediction(s)..")
    df.fillna(0, inplace=True)  ## make sure no NaN exists
    df['prediction'] = model.predict(df[numeric_features + categorical_features])
    
    logger.debug(f" [i] Running evidently report(s)..")    
    report.run(
        reference_data=ref_dataset,
        current_data = df,
        column_mapping = column_mapping
    )

    result = report.as_dict()

    ## metrics[-1] -> last one from report
    ## there's also metrics[0] but not returned
    return result['metrics'][-1]['result']['current']['value']


In [14]:

create_table = """
    drop table if exists metrics;
    create table metrics(
        timestamp date,
        fare_amount float
        )
    """

def prep_database():
    with psycopg.connect(
        dbname='postgres',
        host='localhost',
        port=5432,
        user='postgres',
        password='s3cureP@55w0rd',
        autocommit=True
        ) as conn:
            
        conn.execute(create_table)
    

In [15]:
prep_database()


In [16]:

with psycopg.connect(
    dbname='postgres',
    host='localhost',
    port=5432,
    user='postgres',
    password='s3cureP@55w0rd'
) as conn:
    df = prep_dataset(raw_dataset)
    for this_date in dates:
        
        logger.debug(f" [i] Daily monitoring for date: {str(this_date)}")
        df_chunk = df[ df['lpep_pickup_datetime'].dt.date == this_date ]
        quant = calculate_metric(df_chunk)
        conn.execute(f"""
            INSERT INTO metrics(timestamp, fare_amount) VALUES('{this_date}', {quant});
            """)
        

2024-06-21 12:41:30.948 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-01
2024-06-21 12:41:30.961 | DEBUG    | __main__:calculate_metric:10 -  [i] Running evidently report(s)..
2024-06-21 12:41:30.992 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-02
2024-06-21 12:41:31.003 | DEBUG    | __main__:calculate_metric:10 -  [i] Running evidently report(s)..
2024-06-21 12:41:31.029 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-03
2024-06-21 12:41:31.041 | DEBUG    | __main__:calculate_metric:10 -  [i] Running evidently report(s)..
2024-06-21 12:41:31.068 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-04
2024-06-21 12:41:31.079 | DEBUG    | __main__:calculate_metric:10 -  [i] Running evidently report(s)..
2024-06-21 12:41:31.108 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-05
2024-06-21 12:41:31.121 | DEBUG    | __main__:calculate_metric:10 -  [i] 

2024-06-21 12:41:32.044 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-26
2024-06-21 12:41:32.055 | DEBUG    | __main__:calculate_metric:10 -  [i] Running evidently report(s)..
2024-06-21 12:41:32.089 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-27
2024-06-21 12:41:32.101 | DEBUG    | __main__:calculate_metric:10 -  [i] Running evidently report(s)..
2024-06-21 12:41:32.135 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-28
2024-06-21 12:41:32.147 | DEBUG    | __main__:calculate_metric:10 -  [i] Running evidently report(s)..
2024-06-21 12:41:32.182 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-29
2024-06-21 12:41:32.193 | DEBUG    | __main__:calculate_metric:10 -  [i] Running evidently report(s)..
2024-06-21 12:41:32.227 | DEBUG    | __main__:<module>:11 -  [i] Daily monitoring for date: 2024-03-30
2024-06-21 12:41:32.238 | DEBUG    | __main__:calculate_metric:10 -  [i] 

In [17]:
# query that database
with psycopg.connect(
    dbname='postgres',
    host='localhost',
    port=5432,
    user='postgres',
    password='s3cureP@55w0rd'
) as conn:
    resp = conn.execute("""
        SELECT MAX(fare_amount) FROM metrics;
        """)
    fare_amounts = resp.fetchall()


In [18]:
fare_amounts


[(14.2,)]